In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential

def create_cnn_model(num_conv_layers, num_dense_layers, num_filters=64, kernel_size=(3, 3)):
    # Initialize a Sequential model
    model = Sequential()
    
    # Add convolutional layers
    for i in range(num_conv_layers):
        if i == 0:
            model.add(Conv2D(num_filters, kernel_size, input_shape=(131, 131, 1), activation='relu'))
        else:
            model.add(Conv2D(num_filters, kernel_size, activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Add dropout layer
    model.add(Dropout(0.4))
    
    # Flatten the output of the last convolutional layer
    model.add(Flatten())
    
    # Add dense layers
    for _ in range(num_dense_layers):
        model.add(Dense(256, activation='relu'))
    
    # Output layer
    model.add(Dense(15, activation='softmax'))
    
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

# Define the number of convolutional and dense layers to experiment with
num_conv_layers_list = [3, 4, 5]
num_dense_layers_list = [1, 2, 3, 4]

best_accuracy = 0.0
best_loss = float('inf')
best_model_configuration = None

# Train and evaluate models with different configurations
for num_conv_layers in num_conv_layers_list:
    for num_dense_layers in num_dense_layers_list:
        print(f"Number of Conv Layers: {num_conv_layers}, Number of Dense Layers: {num_dense_layers}")
        
        # Create the model with default values for num_filters and kernel_size
        model = create_cnn_model(num_conv_layers, num_dense_layers)
        
        # Train the model
        model.fit(X_train_preprocessed, y_train, validation_data=(X_val_preprocessed, y_val), epochs = 100, batch_size = 64, callbacks = [Early_Stopping], verbose=0)
        
        # Evaluate the model
        loss, accuracy = model.evaluate(X_test_preprocessed, y_test, verbose=0)
        print("Test Loss: %.4f, Test Accuracy: %.2f%%" % (loss, accuracy * 100)) # Print test accuracy

        # Update best model configuration based on accuracy and lossfe 
        if accuracy > best_accuracy and loss < best_loss:
            best_accuracy = accuracy
            best_loss = loss
            best_model_configuration = (num_conv_layers, num_dense_layers)

print("\nBest Model Configuration - Number of Conv Layers: %d, Number of Dense Layers: %d, Test Loss: %.4f, Test Accuracy: %.2f%%" % (best_model_configuration[0], best_model_configuration[1], best_loss, best_accuracy * 100))import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential

def create_cnn_model(num_conv_layers, num_dense_layers, num_filters=64, kernel_size=(3, 3)):
    # Initialize a Sequential model
    model = Sequential()
    
    # Add convolutional layers
    for i in range(num_conv_layers):
        if i == 0:
            model.add(Conv2D(num_filters, kernel_size, input_shape=(131, 131, 1), activation='relu'))
        else:
            model.add(Conv2D(num_filters, kernel_size, activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Add dropout layer
    model.add(Dropout(0.4))
    
    # Flatten the output of the last convolutional layer
    model.add(Flatten())
    
    # Add dense layers
    for _ in range(num_dense_layers):
        model.add(Dense(256, activation='relu'))
    
    # Output layer
    model.add(Dense(15, activation='softmax'))
    
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

# Define the number of convolutional and dense layers to experiment with
num_conv_layers_list = [3, 4, 5]
num_dense_layers_list = [1, 2, 3, 4]

best_accuracy = 0.0
best_loss = float('inf')
best_model_configuration = None

# Train and evaluate models with different configurations
for num_conv_layers in num_conv_layers_list:
    for num_dense_layers in num_dense_layers_list:
        print(f"Number of Conv Layers: {num_conv_layers}, Number of Dense Layers: {num_dense_layers}")
        
        # Create the model with default values for num_filters and kernel_size
        model = create_cnn_model(num_conv_layers, num_dense_layers)
        
        # Train the model
        model.fit(X_train_preprocessed, y_train, validation_data=(X_val_preprocessed, y_val), epochs = 100, batch_size = 64, callbacks = [Early_Stopping], verbose=0)
        
        # Evaluate the model
        loss, accuracy = model.evaluate(X_test_preprocessed, y_test, verbose=0)
        print("Test Loss: %.4f, Test Accuracy: %.2f%%" % (loss, accuracy * 100)) # Print test accuracy

        # Update best model configuration based on accuracy and lossfe 
        if accuracy > best_accuracy and loss < best_loss:
            best_accuracy = accuracy
            best_loss = loss
            best_model_configuration = (num_conv_layers, num_dense_layers)

print("\nBest Model Configuration - Number of Conv Layers: %d, Number of Dense Layers: %d, Test Loss: %.4f, Test Accuracy: %.2f%%" % (best_model_configuration[0], best_model_configuration[1], best_loss, best_accuracy * 100))import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential

def create_cnn_model(num_conv_layers, num_dense_layers, num_filters=64, kernel_size=(3, 3)):
    # Initialize a Sequential model
    model = Sequential()
    
    # Add convolutional layers
    for i in range(num_conv_layers):
        if i == 0:
            model.add(Conv2D(num_filters, kernel_size, input_shape=(131, 131, 1), activation='relu'))
        else:
            model.add(Conv2D(num_filters, kernel_size, activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Add dropout layer
    model.add(Dropout(0.4))
    
    # Flatten the output of the last convolutional layer
    model.add(Flatten())
    
    # Add dense layers
    for _ in range(num_dense_layers):
        model.add(Dense(256, activation='relu'))
    
    # Output layer
    model.add(Dense(15, activation='softmax'))
    
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

# Define the number of convolutional and dense layers to experiment with
num_conv_layers_list = [3, 4, 5]
num_dense_layers_list = [1, 2, 3, 4]

best_accuracy = 0.0
best_loss = float('inf')
best_model_configuration = None

# Train and evaluate models with different configurations
for num_conv_layers in num_conv_layers_list:
    for num_dense_layers in num_dense_layers_list:
        print(f"Number of Conv Layers: {num_conv_layers}, Number of Dense Layers: {num_dense_layers}")
        
        # Create the model with default values for num_filters and kernel_size
        model = create_cnn_model(num_conv_layers, num_dense_layers)
        
        # Train the model
        model.fit(X_train_preprocessed, y_train, validation_data=(X_val_preprocessed, y_val), epochs = 100, batch_size = 64, callbacks = [Early_Stopping], verbose=0)
        
        # Evaluate the model
        loss, accuracy = model.evaluate(X_test_preprocessed, y_test, verbose=0)
        print("Test Loss: %.4f, Test Accuracy: %.2f%%" % (loss, accuracy * 100)) # Print test accuracy

        # Update best model configuration based on accuracy and lossfe 
        if accuracy > best_accuracy and loss < best_loss:
            best_accuracy = accuracy
            best_loss = loss
            best_model_configuration = (num_conv_layers, num_dense_layers)

print("\nBest Model Configuration - Number of Conv Layers: %d, Number of Dense Layers: %d, Test Loss: %.4f, Test Accuracy: %.2f%%" % (best_model_configuration[0], best_model_configuration[1], best_loss, best_accuracy * 100))